# Insurance Cost Analysis

- Load the data as a pandas dataframe
- Clean the data, taking care of the blank entries
- Run exploratory data analysis (EDA) and identify the attributes that most affect the charges
- Develop single variable and multi variable Linear Regression models for predicting the charges
- Use Ridge regression to refine the performance of Linear regression models.

In [37]:
import pandas as pd
import numpy as np

In [ ]:
file_name = "insurance.csv"

In [49]:
# Set path
df = pd.read_csv("insurance.csv", header=None)

In [50]:
# Print df
print(df.head(10))

    0  1       2  3  4  5            6
0  19  1  27.900  0  1  3  16884.92400
1  18  2  33.770  1  0  4   1725.55230
2  28  2  33.000  3  0  4   4449.46200
3  33  2  22.705  0  0  1  21984.47061
4  32  2  28.880  0  0  1   3866.85520
5  31  1  25.740  0  ?  4   3756.62160
6  46  1  33.440  1  0  4   8240.58960
7  37  1  27.740  3  0  1   7281.50560
8  37  2  29.830  2  0  2   6406.41070
9  60  1  25.840  0  0  1  28923.13692


Data set has no header, so we will add it

In [51]:
# Add headers
headers = ["age", "gender", "bmi", "no_of_children", "smoker", "region", "charges"]
df.columns = headers

In [52]:
# Check df
df

,age,gender,bmi,no_of_children,smoker,region,charges
0,19,1,27.900,0,1,3,16884.92400
1,18,2,33.770,1,0,4,1725.55230
2,28,2,33.000,3,0,4,4449.46200
3,33,2,22.705,0,0,1,21984.47061
4,32,2,28.880,0,0,1,3866.85520
...,...,...,...,...,...,...,...
2767,47,1,45.320,1,0,4,8569.86180
2768,21,1,34.600,0,0,3,2020.17700
2769,19,2,26.030,1,1,1,16450.89470
2770,23,2,18.715,0,0,1,21595.38229


In [53]:
# Check column info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             2772 non-null   object 
 1   gender          2772 non-null   int64  
 2   bmi             2772 non-null   float64
 3   no_of_children  2772 non-null   int64  
 4   smoker          2772 non-null   object 
 5   region          2772 non-null   int64  
 6   charges         2772 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 151.7+ KB


As we can see, age and smoker are considered objects we will have to change that data type to int

In [34]:
# Update data types
df["smoker"] = df["smoker"].astype("int")

ValueError: invalid literal for int() with base 10: '?'

In [54]:
# Update data types
df[["age","smoker"]] = df[["age","smoker"]].astype("int")

ValueError: invalid literal for int() with base 10: '?'

As you can see above when you first try to convert the column values you get an error:

<b>ValueError:<b/> invalid literal for int() with base 10: '?'

so we have to find the column that has missing data, and either remove the rows or add a mean(avg) value to those rows where the data is missing.

In [55]:
# checking for missing values:

null_count = df.isna().sum()
print(null_count)

age               0
gender            0
bmi               0
no_of_children    0
smoker            0
region            0
charges           0
dtype: int64


In [18]:
missing_data = df.isnull()
missing_data.head(5)

,age,gender,bmi,no_of_children,smoker,region,charges
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False


In [56]:
# checking for missing values:

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

age
age
False    2772
Name: count, dtype: int64

gender
gender
False    2772
Name: count, dtype: int64

bmi
bmi
False    2772
Name: count, dtype: int64

no_of_children
no_of_children
False    2772
Name: count, dtype: int64

smoker
smoker
False    2772
Name: count, dtype: int64

region
region
False    2772
Name: count, dtype: int64

charges
charges
False    2772
Name: count, dtype: int64



In [18]:
missing_data = df.isnull()
missing_data.head(5)

,age,gender,bmi,no_of_children,smoker,region,charges
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False


In [57]:
# Temporarily display all rows
with pd.option_context('display.max_rows', None):
    print(df)


     age  gender     bmi  no_of_children smoker  region       charges
0     19       1  27.900               0      1       3  16884.924000
1     18       2  33.770               1      0       4   1725.552300
2     28       2  33.000               3      0       4   4449.462000
3     33       2  22.705               0      0       1  21984.470610
4     32       2  28.880               0      0       1   3866.855200
5     31       1  25.740               0      ?       4   3756.621600
6     46       1  33.440               1      0       4   8240.589600
7     37       1  27.740               3      0       1   7281.505600
8     37       2  29.830               2      0       2   6406.410700
9     60       1  25.840               0      0       1  28923.136920
10    25       2  26.220               0      0       2   2721.320800
11    62       1  26.290               0      1       4  27808.725100
12    23       2  34.400               0      0       3   1826.843000
13    56       1  39

In [24]:
# checking for missing values:

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

age
age
False    2772
Name: count, dtype: int64

gender
gender
False    2772
Name: count, dtype: int64

bmi
bmi
False    2772
Name: count, dtype: int64

no_of_children
no_of_children
False    2772
Name: count, dtype: int64

smoker
smoker
False    2772
Name: count, dtype: int64

region
region
False    2772
Name: count, dtype: int64

charges
charges
False    2772
Name: count, dtype: int64



In [59]:
# Update data types
df["smoker"] = df["smoker"].astype("int")

ValueError: invalid literal for int() with base 10: '?'

In [70]:
#Not ideal, but printed all the rows in the df to see where the error was, row 234 in the age column has a null value (?).
#To fix it, we will add the mean to both columns so we can fill up those null values

In [88]:
# Replace '?' with NaN
df["age"] = df["age"].replace('?', np.nan)

In [89]:
# Convert the age column to float for mean calculation
df["age"] = df["age"].astype('float')

In [90]:
# Calculate the mean of the age column, excluding NaN values
mean_age = df["age"].mean()

In [91]:
# Replace NaN values in the age column with the mean age
df["age"].fillna(mean_age, inplace=True)

C:\Users\dfm_n\AppData\Local\Temp\ipykernel_38404\697676628.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["age"].fillna(mean_age, inplace=True)


In [92]:
# Replace NaN values in the smoker column with the most frequent value
is_smoker = df['smoker'].value_counts().idxmax()
df["smoker"].replace(np.nan, is_smoker, inplace=True)

C:\Users\dfm_n\AppData\Local\Temp\ipykernel_38404\3122606772.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["smoker"].replace(np.nan, is_smoker, inplace=True)


In [93]:
# Convert data types
df[["age", "smoker"]] = df[["age", "smoker"]].astype("int")

In [98]:
# Output to check if the operation was successful
df.dtypes

age                 int32
gender              int64
bmi               float64
no_of_children      int64
smoker              int32
region              int64
charges           float64
dtype: object

In [100]:
# calling df
df

,age,gender,bmi,no_of_children,smoker,region,charges
0,19,1,27.900,0,1,3,16884.92400
1,18,2,33.770,1,0,4,1725.55230
2,28,2,33.000,3,0,4,4449.46200
3,33,2,22.705,0,0,1,21984.47061
4,32,2,28.880,0,0,1,3866.85520
...,...,...,...,...,...,...,...
2767,47,1,45.320,1,0,4,8569.86180
2768,21,1,34.600,0,0,3,2020.17700
2769,19,2,26.030,1,1,1,16450.89470
2770,23,2,18.715,0,0,1,21595.38229


<p>As you can see the bmi and the charges roles have multiple numbers after the column.</p>

<p>We want to creat a pattern and leave only two digts after the decimal.</p>